In [1]:
import syft as sy
import syft.frameworks.torch.nn as nn
import torch as th
import numpy as np

sy.create_sandbox(globals(), False, False)

Setting up Sandbox...
Done!


In [2]:
x_poly = th.zeros(3).polynomial()
y = th.rand(3)
y_poly = y.poly_constant()
z_poly = x_poly + y_poly
out_poly = z_poly * z_poly

In [3]:
x = th.rand(3)
z = x + y
out = (z * z)

In [4]:
factors = out_poly.factors()
vectorized_input = out_poly.poly_vectorize_input_vector(x)
approx_pred = (factors * vectorized_input).sum(1)
approx_pred

tensor([0.7043, 2.4171, 1.1901], dtype=torch.float64)

In [5]:
out

tensor([0.7043, 2.4171, 1.1901])

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 20, bias=True)
        self.fc2 = nn.Linear(20,1, bias=True)

    def forward(self, x):
        x = self.fc1(x)
        x = x * x
        x = self.fc2(x)
        return x

In [30]:
model = Net()
input_data = th.rand((1,10))
true_pred = model(input_data)

In [43]:
_data = input_data.polynomial(minimum_factor=0.001)
poly_pred = model.poly_constant()(_data)

In [44]:
factors = poly_pred.factors()
vectorized_input = poly_pred.poly_vectorize_input_vector(input_data)

In [45]:

approx_pred = (factors * vectorized_input).sum()
approx_pred

tensor(-0.0961, dtype=torch.float64)

In [46]:
true_pred

tensor([[-0.1009]], grad_fn=<AddBackward0>)

In [2]:
input_data = th.rand((1,10))
input_poly = input_data.polynomial() * input_data.poly_constant()

# input_kwargs = get_input_kwargs(input_data)
input_poly

(Wrapper)>PolynomialTensor>[[0.8033546209335327x_0  0.8098037242889404x_1  0.11726242303848267x_2
  0.30903321504592896x_3  0.9365195631980896x_4  0.5178995132446289x_5
  0.7394555807113647x_6  0.5277191400527954x_7  0.3113621473312378x_8
  0.24692851305007935x_9 ]]

In [3]:
input_kwargs = poly_util.get_input_kwargs(input_data)

In [4]:
weights = th.rand(10,20).numpy_tensor() - 0.5
weights2 = th.rand(20,1).numpy_tensor() - 0.5


l1 = input_poly.dot(weights)
l1 = l1 * l1
l2 = l1.dot(weights2)
poly_pred_=l2
poly_pred = poly_pred_#.child.child[0][0]

In [5]:
approx_pred = poly_pred.poly_eval(**input_kwargs)
approx_pred

tensor([-0.4316])

In [13]:
expanded_input_data = l2.poly_vectorize_input_vector(input_data)

In [18]:
expanded_input_data.flatten().dot(factors.flatten())

tensor(-0.4316, dtype=torch.float64)

In [7]:
expanded_input_data = get_expanded_input_vector(input_data, poly_pred)
factors = get_factor_vector(poly_pred)

dot_pred = expanded_input_data.dot(factors)
dot_pred

-0.16864870055045186

In [8]:
l1 = input_data.numpy_tensor().dot(weights)
l1 = l1 * l1
l2 = l1.dot(weights2)
actual_pred = l2.child.child[0][0]
actual_pred

-0.16864859

-0.0056336486731858315

In [90]:


# w2 = th.rand(30, 20).numpy_tensor() - 0.5
# w3 = th.rand(20, 1).numpy_tensor() - 0.5


# l1 = l1 * l1
# l2 = l1.mm(w2)
# l2 = l2 * l2
# l3 = l2.mm(w3)
result = l1.child.child[0][0]

In [96]:
l1_ = input_data.numpy_tensor().mm(w)
# l1_ = l1_ * l1_
# l2_ = l1_.mm(w2)
# l2_ = l2_ * l2_
# l3_ = l2_.mm(w3)

In [97]:
l1_

(Wrapper)>NumpyTensor>[[-0.20451756  0.2684064   0.07124158  0.2902736   0.39842907 -0.07172403
  -0.4367196   0.4091246   0.9582974   0.10398786 -0.21854585  0.02474495
   0.26060078  0.65295804 -0.2139431  -0.3395443   0.3070404  -0.02011314
  -0.561554    0.3085644  -0.2998485  -0.39331004 -0.35617584 -0.32432342
  -0.2411674   0.10843703  0.8528299  -0.06687361 -0.7330738  -0.23397852]]

In [98]:
pred

0.10872250124788474

AttributeError: 'numpy.float64' object has no attribute 'additive_terms'

In [61]:
var2value

{'x_0': 0.8706938624382019,
 'x_1': 0.6152476072311401,
 'x_2': 0.7849135398864746,
 'x_3': 0.2315431833267212,
 'x_4': 0.5164133906364441,
 'x_5': 0.9556822180747986,
 'x_6': 0.1679670810699463,
 'x_7': 0.6041491627693176,
 'x_8': 0.027327358722686768,
 'x_9': 0.5556422472000122}

In [34]:
from collections import Counter

In [41]:
cnt = Counter()
for t in result.additive_terms:
    cnt[int(abs(t.factor) * 1000)] += 1

In [4]:
%%timeit
result = vt.mm(w)

1.14 s ± 21.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
vt = var_tensor((200, 500))
w = th.ones(200,500).numpy_tensor() + 2

In [6]:
%%timeit
result = vt * w

724 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
import multiprocessing
sy._pool = multiprocessing.Pool(32)

In [58]:
%%timeit

inputs = list(zip(vt.child.child, w.child.child))
results = sy.pool().map(multiply, inputs)

804 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
import syft as sy

In [ ]:
sy.pool()

In [47]:
def multiply(left_and_right):
    left, right = left_and_right
    return left * right

In [35]:
# inputs[0]

In [8]:
result.shape

(100, 100)

1.1 s ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


861 ms ± 4.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
result_ = vt.child.child.dot(w.child.child)

2.94 s ± 51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
result.shape

(50, 40)

In [8]:
result_.shape

(50, 40)

In [5]:
raw_result.shape

(7, 5, 4)

In [32]:
other.shape

(7, 5, 4)

In [27]:
self.child.child.shape

(7, 5, 4)

In [17]:
self.shape

(7, 5, 1)

In [18]:
other.shape

(1, 5, 4)

(7, 1, 5)

In [28]:
out = vt.dot(w)

In [30]:
out.shape

(7, 4)

In [121]:
orig_shape = vt.shape
print(orig_shape)



(5, 9, 2)


In [114]:
sum1(vt.child.child)

array([x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 ,
       x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 ,
       x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 ,
       x_27 + x_28 + x_29 + x_30 + x_31 + x_32 + x_33 + x_34 + x_35 ,
       x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 + x_44 ,
       x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 ,
       x_54 + x_55 + x_56 + x_57 + x_58 + x_59 + x_60 + x_61 + x_62 ,
       x_63 + x_64 + x_65 + x_66 + x_67 + x_68 + x_69 + x_70 + x_71 ,
       x_72 + x_73 + x_74 + x_75 + x_76 + x_77 + x_78 + x_79 + x_80 ,
       x_81 + x_82 + x_83 + x_84 + x_85 + x_86 + x_87 + x_88 + x_89 ,
       x_90 + x_91 + x_92 + x_93 + x_94 + x_95 + x_96 + x_97 + x_98 ,
       x_99 + x_100 + x_101 + x_102 + x_103 + x_104 + x_105 + x_106 + x_107 ,
       x_108 + x_109 + x_110 + x_111 + x_112 + x_113 + x_114 + x_115 + x_116 ],
      dtype=object)

In [107]:
gold = vt.sum(1).child.child
gold

array([x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 ,
       x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 ,
       x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 ,
       x_27 + x_28 + x_29 + x_30 + x_31 + x_32 + x_33 + x_34 + x_35 ,
       x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 + x_44 ,
       x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 ,
       x_54 + x_55 + x_56 + x_57 + x_58 + x_59 + x_60 + x_61 + x_62 ,
       x_63 + x_64 + x_65 + x_66 + x_67 + x_68 + x_69 + x_70 + x_71 ,
       x_72 + x_73 + x_74 + x_75 + x_76 + x_77 + x_78 + x_79 + x_80 ,
       x_81 + x_82 + x_83 + x_84 + x_85 + x_86 + x_87 + x_88 + x_89 ,
       x_90 + x_91 + x_92 + x_93 + x_94 + x_95 + x_96 + x_97 + x_98 ,
       x_99 + x_100 + x_101 + x_102 + x_103 + x_104 + x_105 + x_106 + x_107 ,
       x_108 + x_109 + x_110 + x_111 + x_112 + x_113 + x_114 + x_115 + x_116 ],
      dtype=object)

In [91]:
pred = sum0(vt)

In [36]:
%%timeit
vt.sum(1)

1.11 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
vt = var_tensor((1,28*28))

dims = 512
weights = th.ones(28*28, dims).numpy_tensor().child.child * 2
raw = vt.child.child
expanded_raw = np.repeat(raw,dims).reshape(28*28, dims)

In [55]:
out = (expanded_raw * weights)

In [62]:
%%timeit
out[:,0].sum()

938 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
out.shape

(784, 512)

In [5]:
out = vt.mm(weights)

In [11]:
%%timeit
out = vt.mm(weights)

2.19 s ± 78.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [120]:
%%timeit
out = vt.mm(weights)

2.27 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
weights.child.child = weights.child.child.astype("float")

In [80]:
vt

(Wrapper)>NumpyTensor>[['x_0' 'x_1' 'x_2' 'x_3' 'x_4' 'x_5' 'x_6' 'x_7' 'x_8' 'x_9']]

In [81]:
vt + weights

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [46]:
vt.dot(weights)

TypeError: can't multiply sequence by non-int of type 'float'

In [7]:
print("\tau^{e}_{1}")

	au^{e}_{1}


In [2]:
v = list()
for i in range(10):
    v.append(poly.var("x_" + str(i)))
v = np.array(v)

In [3]:
y = v + v

In [4]:
out = y.dot(np.ones(10))

In [11]:
import sympy

var = poly.var

def evaluate_poly(x_factor, x_power, y_factor, y_power, z_factor, z_power):
    
    x,y,z = var("x"), var("y"), var("z")
    poly = ((x ** x_power) * x_factor) + ((y ** y_power) * y_factor) + ((z ** z_power) * z_factor)
    syft_results = list()
    for xi in range(-10,10):
        for yi in range(-10,10):
            for zi in range(-10,10):            
                result = poly(x=zi, y=yi, z=zi)
                syft_results.append(result)
                
                
    x = sympy.poly(sympy.symbols('x'))
    y = sympy.poly(sympy.symbols('y'))
    z = sympy.poly(sympy.symbols('z'))

    poly = ((x ** x_power) * x_factor) + ((y ** y_power) * y_factor) + ((z ** z_power) * z_factor)
    np_results = list()
    for xi in range(-10,10):
        for yi in range(-10,10):
            for zi in range(-10,10):            
                result = poly(zi, yi, zi)
                np_results.append(result)

    return np.array(syft_results), np.array(np_results)

def test_raw_poly():

    import random
    random.seed(0)
    for i in range(10):
        a = random.randrange(-1000, 1000) / 100
        b = random.randrange(0, 10)
        c = random.randrange(-1000, 1000) / 100
        d = random.randrange(0, 10)
        e = random.randrange(-1000, 1000) / 100
        f = random.randrange(0, 10)
        s,n = evaluate_poly(a, b, c, d, e, f)
        assert (np.abs(s - n).max()) < 0.000001

In [12]:
test_raw_poly()

In [188]:
s - n

array([-8.75000011920929, -8.75000000000000, -8.75000000000000, ...,
       -8.75000000000000, -8.75000000000000, -8.75000000000000],
      dtype=object)

In [187]:
n

array([639910008.750000, 274643403.140000, 106595623.470000, ...,
       36404138.8200000, 106595623.470000, 274643403.140000], dtype=object)

In [114]:
print(poly)
poly(x=3, y=2, z=2)

1x + 1y + 1z 


7

In [73]:
import sympy

33

33